In [21]:
import pandas as pd
from pathlib import Path
import pathlib

import logging
import getpass
import os
import sys
import shutil
import numpy as np

user = getpass.getuser()

Notebook to maintain BAUS visualizer output files. Core idea is to set a list of runs, and then have the script handle the moving of the relevant files into a staging and production directory.

# Constants and paths

In [2]:
LOG_FILENAME = 'viz_dir_file_logger.log'

In [ ]:
# in my (AO) case, M:\urban_modeling is mounted to /Volumes/Data/Models/urban_modeling

HOME_DIR = pathlib.Path.home()
M_DRIVE = Path("/Volumes/Data/Models") if os.name != "nt" else "M:/"
BOX_DIR = HOME_DIR / 'Box'

In [26]:
# build relevant dirs from the stubs

# the "inactive" location for files not to be used for the viz

viz_dir_storage = M_DRIVE / 'urban_modeling' / 'baus' / \
    'PBA50Plus' / 'viz_test' / 'STORAGE_BAUS_Visualizer_PBA50Plus_Files'


# the active location for files to be used for the viz (consistent with model_run_inventory.csv)
viz_dir_prod = M_DRIVE / 'urban_modeling' / 'baus' / \
    'PBA50Plus' / 'viz_test' / 'PROD_BAUS_Visualizer_PBA50Plus_Files'

# a temporary overflow dir where files are moved to instead of deleted
viz_dir_overflow = M_DRIVE / 'urban_modeling' / 'baus' / \
    'PBA50Plus' / 'viz_test' / 'OVERFLOW'

In [5]:
# main inventory - these are updated based on the list in this notebook

model_run_inventory_path = M_DRIVE / 'urban_modeling' / 'baus' / \
    'PBA50Plus' / 'BAUS_Visualizer_PBA50Plus_Files' / 'model_run_inventory.csv'

# test path
model_run_inventory_test_path = M_DRIVE / 'urban_modeling' / 'baus' / \
    'PBA50Plus' / 'viz_test' / 'PROD_BAUS_Visualizer_PBA50Plus_Files' / \
    'model_run_inventory.csv'

In [6]:
# create logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
# console handler
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(logging.Formatter(
    '%(asctime)s - %(levelname)s - %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p'))
logger.addHandler(ch)
# file handler - info
fh = logging.FileHandler(viz_dir_prod / LOG_FILENAME.format("info"), mode='w')
fh.setLevel(logging.INFO)
fh.setFormatter(logging.Formatter(
    '%(asctime)s - %(levelname)s - %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p'))
logger.addHandler(fh)
# file handler - debug
fh = logging.FileHandler(viz_dir_prod / LOG_FILENAME.format("debug"), mode='w')
fh.setLevel(logging.DEBUG)
fh.setFormatter(logging.Formatter(
    '%(asctime)s - %(levelname)s - %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p'))
logger.addHandler(fh)

In [7]:
def m_swap(path):
    if path[:3] in ["M:/", "M:\\"]:
        new_path = M_DRIVE / path[3:]
        return new_path
    else:
        return Path(path)

## Run log loading

In [8]:
# this file is maintained by 
# https://github.com/BayAreaMetro/bayarea_urbansim/blob/metrics_update/scripts/create_run_log_from_yamls.py
# reading off run_setup.yaml files

run_log_path = M_DRIVE /\
    "urban_modeling" / "baus" / "PBA50Plus" / "run_setup_tracker_autogen.csv"
run_log_path

PosixPath('/Volumes/Data/Models/urban_modeling/baus/PBA50Plus/run_setup_tracker_autogen.csv')

In [9]:
run_log = pd.read_csv(run_log_path, parse_dates=[
                      'time_stamp_start_log', 'time_stamp_end_log', 'time_stamp_start_pth'])

run_log = run_log.sort_values('time_stamp_start_log', ascending=False)
run_log.head(2)

,yaml_path,is_complete,time_stamp_start_log,time_stamp_start_pth,time_stamp_end_log,run_name,inputs_dir,outputs_dir,run_simulation_validation,run_summaries,...,use_pipeline_filters,run_jobs_to_transit_strategy_elcm,run_jobs_to_transit_strategy_random,jobs_to_transit_strategy_random_reloc_rate,variant,vintage,reference_run,annotation,renter_protections_file,presevervation_file
38,/Volumes/Data/Models/urban_modeling/baus/PBA50...,True,2024-05-16 22:39:13,2024-05-16 22:39:13,2024-05-17 06:34:17,PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_re...,M:/urban_modeling/baus/BAUS Inputs,M:/urban_modeling/baus/PBA50Plus/PBA50Plus_Dra...,True,True,...,False,True,False,0.25,DBP,PBA2050Plus,PBA50Plus_Draft_Blueprint_v6,"['Deed restriction fix', 'H6 pipeline fix', 'T...",NaN,NaN
36,/Volumes/Data/Models/urban_modeling/baus/PBA50...,True,2024-05-16 22:38:03,2024-05-16 22:38:03,2024-05-17 06:39:50,PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_al...,M:/urban_modeling/baus/BAUS Inputs,M:/urban_modeling/baus/PBA50Plus/PBA50Plus_Dra...,True,True,...,False,True,False,0.25,DBP,PBA2050Plus,PBA50Plus_Draft_Blueprint_v6,"['Deed restriction fix', 'H6 pipeline fix', 'T...",NaN,NaN


## Run log BAUS path prep

We use the log file to add fully qualified paths to each relevant file for the visualizer.

In [10]:
def runlog_path_creator(run_log, years=[2020, 2050, 'growth']):

    ## CORE SUMMARIES ##
    run_log['core_summaries_path'] = run_log.apply(
        lambda x: f'{x.outputs_dir}/{x.run_name}/core_summaries', axis=1).map(m_swap)

    # buildings
    for year in years:
        run_log[f'buildings_path_{year}'] = run_log.apply(
            lambda x: f'{x.outputs_dir}/{x.run_name}/core_summaries/{x.run_name}_building_summary_{year}.csv', axis=1).map(m_swap)

    # parcels
    for year in years:
        run_log[f'parcels_path_{year}'] = run_log.apply(
            lambda x: f'{x.outputs_dir}/{x.run_name}/core_summaries/{x.run_name}_parcel_summary_{year}.csv', axis=1).map(m_swap)

    # interim zones
    for year in years:
        allyears = 'allyears' if year == 'growth' else year

        run_log[f'zone_interim_path_{year}'] = run_log.apply(
            lambda x: f'{x.outputs_dir}/{x.run_name}/core_summaries/{x.run_name}_interim_zone_output_{allyears}.csv', axis=1).map(m_swap)

    # redevelopment
    run_log[f'redev_path'] = run_log.apply(
        lambda x: f'{x.outputs_dir}/{x.run_name}/redevelopment_summaries/{x.run_name}_county_redev_summary_growth.csv', axis=1).map(m_swap)

    #  zones
    for year in years:
        run_log[f'zone_path_{year}'] = run_log.apply(
            lambda x: f'{x.outputs_dir}/{x.run_name}/travel_model_summaries/{x.run_name}_taz1_summary_{year}.csv', axis=1).map(m_swap)

    # GEOGRAPHIC SUMMARIES
    # superdistricts

    for year in years:
        for geo in ['county', 'superdistrict', 'juris']:
            run_log[f'{geo}_path_{year}'] = run_log.apply(
                lambda x: f'{x.outputs_dir}/{x.run_name}/geographic_summaries/{x.run_name}_{geo}_summary_{year}.csv', axis=1).map(m_swap)

# hack - renamed a run name after the fact after realizing settings and name inconsistent
# our path generation would not work for these runs since the output path based on the wrong name in the yaml is now obsolete

run_name_fix_old = 'PBA50Plus_Draft_Blueprint_v3_exog_en7r'
run_name_fix_new = 'PBA50Plus_Draft_Blueprint_v3_no_exog_no_adj'

# change name back to original name so paths will work
run_log.loc[run_log.run_name == run_name_fix_new,
            'run_name'] = run_name_fix_old

# run the paths on the run_log inventory
runlog_path_creator(run_log)

# reverse fix after adding the paths
run_log.loc[run_log.run_name == run_name_fix_old,
            'run_name'] = run_name_fix_new

In [11]:
# run_log_subset = run_log.query('time_stamp_start_log>"2024-04-12" & is_complete ')
# run_log_subset.shape

# Define functions for getting a list of files and moving them between storage and production dirs

In [22]:
def write_list_to_file(data, path):
    """
    Writes a list of strings to a file, with each string on a new line, 
    and includes a header line at the top.

    Parameters:
    data (list of str): The list of strings to write to the file.
    path (str): The file path where the data will be written.

    """
    with open(path, 'w') as file:
        # add header
        file.write("run_name\n")
        # add a run per line
        for item in data:
            file.write(f"{item}\n")

In [25]:
def generate_visualizer_files_from_run_name(run_names, viz_dir_prod, viz_dir_storage):
    """
    Generates a dictionary of file paths for visualizer components based on run names 
    and storage directories, and logs the number of runs processed.

    Parameters:
    run_names (list of str): List of run names to generate file paths for.
    viz_dir_prod (Path): Path to the production visualizer directory.
    viz_dir_storage (Path): Path to the storage visualizer directory.

    Returns:
    dict: A nested dictionary where the first level keys are run names, 
          the second level keys are component names, and the third level keys 
          are storage types ('PROD' or 'STORAGE'), each mapping to the respective file paths.

    Example:
    >>> run_names = ['run1', 'run2']
    >>> viz_dir_prod = Path('/path/to/prod')
    >>> viz_dir_storage = Path('/path/to/storage')
    >>> result = generate_visualizer_files_from_run_name(run_names, viz_dir_prod, viz_dir_storage)
    This will return a dictionary structured as follows:
    {
        'run1': {
            'new_buildings': {'PROD': Path('/path/to/prod/run1_new_buildings_summary.csv'),
                              'STORAGE': Path('/path/to/storage/run1_new_buildings_summary.csv')},
            ...
        },
        'run2': {
            'new_buildings': {'PROD': Path('/path/to/prod/run2_new_buildings_summary.csv'),
                              'STORAGE': Path('/path/to/storage/run2_new_buildings_summary.csv')},
            ...
        }
    }
    """
    #
    # Initialize the outermost dictionary to hold all run data
    l_0 = {}

    for run_name in run_names:
        # Define the paths for various components for the current run
        component_paths = {
            "new_buildings": f"{run_name}_new_buildings_summary.csv",
            "taz": f"{run_name}_taz1_summary_growth.csv",
            "interim_zone_output": f"{run_name}_interim_zone_output_allyears.csv",

            "juris_dr": f"{run_name}_juris_dr_growth.csv",
            "county_dr": f"{run_name}_county_dr_growth.csv",
            "superdistrict_dr": f"{run_name}_superdistrict_dr_growth.csv",

            "juris_summary": f"{run_name}_juris_summary_growth.csv",
            "county_summary": f"{run_name}_county_summary_growth.csv",
            "superdistrict_summary": f"{run_name}_superdistrict_summary_growth.csv"
        }

        # Storage types and their corresponding directories
        storage_type = {'PROD': viz_dir_prod, 'STORAGE': viz_dir_storage}

        # Initialize the dictionary for the current run's components
        l_1 = {}
        for component_key, component_value in component_paths.items():
            # Initialize the dictionary for the storage locations of the current component
            l_2 = {}
            for storage_key, storage_value in storage_type.items():
                # Combine the storage path with the component file name
                this_combo = storage_value / component_value
                l_2[storage_key] = this_combo  # Map storage type to file path

            # Map component name to its storage locations
            l_1[component_key] = l_2

        # Map run name to its components and their storage locations
        l_0[run_name] = l_1

    logging.info(f'Generated summary files for {len(l_0)} runs')
    return l_0

In [27]:
def clear_prod_files(run_names, viz_dir_prod, viz_dir_storage, viz_dir_overflow):
    """
    Moves CSV files from the production directory to the storage directory, 
    excluding the 'model_run_inventory' file, and handles file management 
    to ensure no duplicates are left in the production directory.

    Parameters:
    run_names (list of str): List of run names to generate the desired run files.
    viz_dir_prod (Path): Path to the production visualizer directory.
    viz_dir_storage (Path): Path to the storage visualizer directory.
    viz_dir_overflow (Path): Path to the overflow directory for duplicate files.

    """
    # Step 0: Get desired run files
    desired_run_files = generate_visualizer_files_from_run_name(
        run_names, viz_dir_prod, viz_dir_storage)

    logging.info(
        f'Found {len(desired_run_files)} from {len(run_names)}')

    # Move all CSVs from PROD to STORAGE (keeping the inventory)
    prod_files = viz_dir_prod.rglob('*.csv')
    for f in prod_files:
        if 'model_run_inventory' not in f.name:
            target_path = viz_dir_storage / f.name

            try:
                if not target_path.exists():
                    logging.info(
                        f'MOVING to STORAGE:\n\t{f.name}')
                    shutil.move(f, viz_dir_storage)
                else:
                    # shutil.move throws an error if there is already an identically named file at the storage location
                    # we can probably just delete these - but for now, we move to overflow 

                    # Check if sizes are the same
                    if f.stat().st_size == target_path.stat().st_size:
                        # If size and name are the same, assume they are identical and move to OVERFLOW
                        logging.info(
                            f'File already in storage; moving to OVERFLOW - probably safe to delete\n\t{f.name}')
                        shutil.move(f, viz_dir_overflow)
            except FileNotFoundError:
                logging.error(f'File not found: {f}')
            except OSError as e:
                logging.error(f'Error moving file {f}: {e}')

In [28]:
def move_desired_run_files(run_names, viz_dir_prod, viz_dir_storage):
    """
    Moves desired run files from the storage directory to the production directory 
    if they do not already exist in the production directory.

    Parameters:
    run_names (list of str): List of run names to generate the desired run files.
    viz_dir_prod (Path): Path to the production visualizer directory.
    viz_dir_storage (Path): Path to the storage visualizer directory.

    """
    # Get desired run files
    desired_run_files = generate_visualizer_files_from_run_name(run_names, viz_dir_prod, viz_dir_storage)

    for run_name, component_dict in desired_run_files.items():
        for component, paths in component_dict.items():
            try:
                # Check if the file exists in the production directory
                if not paths["PROD"].exists():
                    logging.info(
                        f'File: {paths["STORAGE"].name} is not found in PROD; moving from STORAGE'
                    )
                    shutil.move(paths["STORAGE"], paths["PROD"])
                else:
                    logging.info(
                        f'File: {paths["STORAGE"].name} is already found in PROD; nothing to be done'
                    )
            except FileNotFoundError:
                logging.debug(f'Not found: {paths["STORAGE"].name}')
            except OSError as e:
                logging.debug(
                    f'File cannot be moved or accessed: {paths["STORAGE"].name}, Error: {e}'
                )


# Identify key run parameters (run lists and paths)



## Curate lists with runs to stage for the visualizer

In [29]:
# manual curation of runs to include

pba50_runs = ['PBA50_FinalBlueprint_Exogenous_v2', 'PBA50_FBP',  # PBA50_FBP2020
              'PBA50_NoProject_Exogenous',
              # 'PBA50_FBP2015',
              'PBA50_NP',
              # 'PBA50_NP2015'
              ]

pba50p_runs = [
               'PBA50Plus_DBP_InitialRun_v7',
               'PBA50Plus_NP_InitialRun_v8',
               'PBA50Plus_NoProject_v7',
               'PBA50Plus_NoProject_v9',
               'PBA50Plus_NoProject_v10_zn_znmod_upd',
               'PBA50Plus_NoProject_v11',
               'PBA50Plus_Draft_Blueprint_v6',
               'PBA50Plus_Draft_Blueprint_v7_znupd',
               'PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix',
               'PBA50Plus_Draft_Blueprint_v10_znupd_nodevfix_noexog',
               'PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_altseed',
               'PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_altseed_v2',
               'PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_reloc_adj'
               ]

# add any additional runs in current metrics inventury

# metrics_inventory_df = pd.read_csv(metrics_model_run_inventory_path)
# current_metrics_runs = metrics_inventory_df.directory.str.split('/').map(lambda x: x[-1])

# additional_runs_from_metrics_inventory = list(set(current_metrics_runs) - set(pba50p_runs+pba50_runs))

multiple_runs = {'run_names': pba50_runs + pba50p_runs,  # + additional_runs_from_metrics_inventory,
                 'viz_dir_prod': viz_dir_prod, 'viz_dir_storage': viz_dir_storage}
multiple_runs

{'run_names': ['PBA50_FinalBlueprint_Exogenous_v2',
  'PBA50_FBP',
  'PBA50_NoProject_Exogenous',
  'PBA50_NP',
  'PBA50Plus_DBP_InitialRun_v7',
  'PBA50Plus_NP_InitialRun_v8',
  'PBA50Plus_NoProject_v7',
  'PBA50Plus_NoProject_v9',
  'PBA50Plus_NoProject_v10_zn_znmod_upd',
  'PBA50Plus_NoProject_v11',
  'PBA50Plus_Draft_Blueprint_v6',
  'PBA50Plus_Draft_Blueprint_v7_znupd',
  'PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix',
  'PBA50Plus_Draft_Blueprint_v10_znupd_nodevfix_noexog',
  'PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_altseed',
  'PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_altseed_v2',
  'PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_reloc_adj'],
 'viz_dir_prod': PosixPath('/Volumes/Data/Models/urban_modeling/baus/PBA50Plus/viz_test/PROD_BAUS_Visualizer_PBA50Plus_Files'),
 'viz_dir_storage': PosixPath('/Volumes/Data/Models/urban_modeling/baus/PBA50Plus/viz_test/STORAGE_BAUS_Visualizer_PBA50Plus_Files')}

# Run the funcions

In [17]:
# write out run inventory for use in visualiser

#write_list_to_file(multiple_runs['run_names'], model_run_inventory_path)
write_list_to_file(multiple_runs['run_names'], model_run_inventory_path)


In [19]:
# step CLEARS out production viz files, moving them to storage

clear_prod_files(**multiple_runs)

05/17/2024 11:34:23 AM - INFO - Generated summary files for 17 runs
05/17/2024 11:34:23 AM - INFO - Found 17 from 17
05/17/2024 11:34:23 AM - INFO - MOVING to STORAGE:
	PBA50Plus_2020Validation_GG_Pipeline_Strategies_county_dr_growth.csv
05/17/2024 11:34:23 AM - INFO - MOVING to STORAGE:
	PBA50Plus_2020Validation_GG_Pipeline_Strategies_county_summary_growth.csv
05/17/2024 11:34:23 AM - INFO - MOVING to STORAGE:
	PBA50Plus_2020Validation_GG_Pipeline_Strategies_interim_zone_output_allyears.csv
05/17/2024 11:34:24 AM - INFO - MOVING to STORAGE:
	PBA50Plus_2020Validation_GG_Pipeline_Strategies_juris_dr_growth.csv
05/17/2024 11:34:24 AM - INFO - MOVING to STORAGE:
	PBA50Plus_2020Validation_GG_Pipeline_Strategies_juris_summary_growth.csv
05/17/2024 11:34:24 AM - INFO - MOVING to STORAGE:
	PBA50Plus_2020Validation_GG_Pipeline_Strategies_new_buildings_summary.csv
05/17/2024 11:34:24 AM - INFO - MOVING to STORAGE:
	PBA50Plus_2020Validation_GG_Pipeline_Strategies_superdistrict_dr_growth.csv
05/1

05/17/2024 11:35:19 AM - INFO - MOVING to STORAGE:
	PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_reloc_adj_county_summary_growth.csv
05/17/2024 11:35:19 AM - INFO - MOVING to STORAGE:
	PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_reloc_adj_interim_zone_output_allyears.csv
05/17/2024 11:35:19 AM - INFO - MOVING to STORAGE:
	PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_reloc_adj_juris_dr_growth.csv
05/17/2024 11:35:20 AM - INFO - MOVING to STORAGE:
	PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_reloc_adj_juris_summary_growth.csv
05/17/2024 11:35:20 AM - INFO - MOVING to STORAGE:
	PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_reloc_adj_new_buildings_summary.csv
05/17/2024 11:35:21 AM - INFO - MOVING to STORAGE:
	PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_reloc_adj_superdistrict_dr_growth.csv
05/17/2024 11:35:23 AM - INFO - MOVING to STORAGE:
	PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_reloc_adj_superdistrict_summary_growth.csv
05/17/2024 11:35:24 AM - INFO - MOVING to STORAGE:
	PBA50Plus_Draft

05/17/2024 11:36:18 AM - INFO - MOVING to STORAGE:
	PBA50Plus_NoProject_v13_zn_revisit_ugb_superdistrict_summary_growth.csv
05/17/2024 11:36:18 AM - INFO - MOVING to STORAGE:
	PBA50Plus_NoProject_v13_zn_revisit_ugb_taz1_summary_growth.csv
05/17/2024 11:36:18 AM - INFO - MOVING to STORAGE:
	PBA50Plus_NoProject_v13_zn_revisit_ugb_w_ubi_county_dr_growth.csv
05/17/2024 11:36:19 AM - INFO - MOVING to STORAGE:
	PBA50Plus_NoProject_v13_zn_revisit_ugb_w_ubi_county_summary_growth.csv
05/17/2024 11:36:23 AM - INFO - MOVING to STORAGE:
	PBA50Plus_NoProject_v13_zn_revisit_ugb_w_ubi_interim_zone_output_allyears.csv
05/17/2024 11:36:24 AM - INFO - MOVING to STORAGE:
	PBA50Plus_NoProject_v13_zn_revisit_ugb_w_ubi_juris_dr_growth.csv
05/17/2024 11:36:24 AM - INFO - MOVING to STORAGE:
	PBA50Plus_NoProject_v13_zn_revisit_ugb_w_ubi_juris_summary_growth.csv
05/17/2024 11:36:24 AM - INFO - MOVING to STORAGE:
	PBA50Plus_NoProject_v13_zn_revisit_ugb_w_ubi_new_buildings_summary.csv
05/17/2024 11:36:24 AM - INF

In [20]:
# step MOVES files out of storage and back to production folder
# consider wrapping these into one step calling both

move_desired_run_files(**multiple_runs)

05/17/2024 11:37:10 AM - INFO - Generated summary files for 17 runs
05/17/2024 11:37:10 AM - INFO - File: PBA50_FinalBlueprint_Exogenous_v2_new_buildings_summary.csv is not found in PROD; moving from STORAGE
05/17/2024 11:37:10 AM - INFO - File: PBA50_FinalBlueprint_Exogenous_v2_taz1_summary_growth.csv is not found in PROD; moving from STORAGE
05/17/2024 11:37:10 AM - INFO - File: PBA50_FinalBlueprint_Exogenous_v2_interim_zone_output_allyears.csv is not found in PROD; moving from STORAGE
05/17/2024 11:37:10 AM - INFO - File: PBA50_FinalBlueprint_Exogenous_v2_juris_dr_growth.csv is not found in PROD; moving from STORAGE
05/17/2024 11:37:10 AM - INFO - File: PBA50_FinalBlueprint_Exogenous_v2_county_dr_growth.csv is not found in PROD; moving from STORAGE
05/17/2024 11:37:11 AM - INFO - File: PBA50_FinalBlueprint_Exogenous_v2_superdistrict_dr_growth.csv is not found in PROD; moving from STORAGE
05/17/2024 11:37:11 AM - INFO - File: PBA50_FinalBlueprint_Exogenous_v2_juris_summary_growth.csv

05/17/2024 11:37:46 AM - INFO - File: PBA50Plus_NoProject_v9_taz1_summary_growth.csv is not found in PROD; moving from STORAGE
05/17/2024 11:37:47 AM - INFO - File: PBA50Plus_NoProject_v9_interim_zone_output_allyears.csv is not found in PROD; moving from STORAGE
05/17/2024 11:37:47 AM - INFO - File: PBA50Plus_NoProject_v9_juris_dr_growth.csv is not found in PROD; moving from STORAGE
05/17/2024 11:37:47 AM - INFO - File: PBA50Plus_NoProject_v9_county_dr_growth.csv is not found in PROD; moving from STORAGE
05/17/2024 11:37:47 AM - INFO - File: PBA50Plus_NoProject_v9_superdistrict_dr_growth.csv is not found in PROD; moving from STORAGE
05/17/2024 11:37:47 AM - INFO - File: PBA50Plus_NoProject_v9_juris_summary_growth.csv is not found in PROD; moving from STORAGE
05/17/2024 11:37:47 AM - INFO - File: PBA50Plus_NoProject_v9_county_summary_growth.csv is not found in PROD; moving from STORAGE
05/17/2024 11:37:54 AM - INFO - File: PBA50Plus_NoProject_v9_superdistrict_summary_growth.csv is not f

05/17/2024 11:38:30 AM - INFO - File: PBA50Plus_Draft_Blueprint_v10_znupd_nodevfix_noexog_juris_summary_growth.csv is not found in PROD; moving from STORAGE
05/17/2024 11:38:30 AM - INFO - File: PBA50Plus_Draft_Blueprint_v10_znupd_nodevfix_noexog_county_summary_growth.csv is not found in PROD; moving from STORAGE
05/17/2024 11:38:33 AM - INFO - File: PBA50Plus_Draft_Blueprint_v10_znupd_nodevfix_noexog_superdistrict_summary_growth.csv is not found in PROD; moving from STORAGE
05/17/2024 11:38:35 AM - INFO - File: PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_altseed_new_buildings_summary.csv is not found in PROD; moving from STORAGE
05/17/2024 11:38:35 AM - INFO - File: PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_altseed_taz1_summary_growth.csv is not found in PROD; moving from STORAGE
05/17/2024 11:38:36 AM - INFO - File: PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_altseed_interim_zone_output_allyears.csv is not found in PROD; moving from STORAGE
05/17/2024 11:38:36 AM - INFO - File: PBA

In [56]:
# run_log[run_log.household_controls_file=='household_controls_PBA50.csv']